In [97]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [259]:
dataset = pd.read_csv("heart.csv")
dataset.target.value_counts()

1    165
0    138
Name: target, dtype: int64

In [260]:
X = dataset.drop('target', axis=1)
y = dataset.get('target')

In [261]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0,train_size=0.7)

In [262]:
y_train.values.reshape(1,y_train.values.shape[0]).shape

(1, 212)

In [263]:
X_train = X_train.T
y_train = y_train.values.reshape(1,y_train.values.shape[0])
X_test = X_test.T
y_test = y_test.values.reshape(1,y_test.values.shape[0])

In [255]:
y_train.shape

(1, 212)

In [266]:
def initialize_parameters(layer_dims):
    np.random.seed(0)
    l = len(layer_dims)
    parameters = {}
    L = len(layer_dims)
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
    return parameters

In [267]:
model_parameters = initialize_parameters([X_train.shape[0],100,60,1])

In [268]:
model_parameters['W1'].shape

(100, 13)

In [155]:
len(model_parameters)

6

In [109]:
def forward_propagation(A_prev,W,b,activation):
    Z = np.dot(W,A_prev)+b
    if activation == "sigmoid":
        A = 1/(1+np.exp(-Z))
    if activation == "tanh":
        A = np.tanh(Z)
    if activation == "relu":
        A = np.maximum(Z,0)
    linear_cache = (A, W, b)
    activation_cache = Z
    cache = (linear_cache, activation_cache)
    return A,cache

In [112]:
def model_forward(X,parameters):
    caches = []
    A = X
    L = len(parameters) // 2
    for l in range(1, L):
        A_prev = A
        A, cache = forward_propagation(A_prev,parameters["W"+str(l)],parameters["b"+str(l)],"relu")
        caches.append(cache)
    AL, cache = forward_propagation(A,parameters["W"+str(L)],parameters["b"+str(L)],"sigmoid")
    caches.append(cache)
    return AL,caches

In [115]:
def compute_cost(AL,Y):
    m = Y.shape[1]
    cost = -np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))/m
    return cost

In [116]:
def sigmoid_backward(dA, cache):
    Z = cache 
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    return dZ

In [117]:
def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    return dZ

In [199]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = 1./m * np.dot(dZ, A_prev.T)
    db = 1./m * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    
    return dA_prev, dW, db

In [200]:
def linear_activation_backward(dA, cache, activation):

    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, cache[1])
        dA_prev, dW, db = linear_backward(dZ, cache[0])
   
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, cache[1])
        dA_prev, dW, db = linear_backward(dZ, cache[0])
    
    return dA_prev, dW, db

In [201]:
cost = compute_cost(AL,y_train.values.reshape(1,212))
cost

0.6917056958141469

In [204]:
def model_backward(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    dAL = -(np.divide(Y,AL)-np.divide(1-Y,1-AL))
    
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL,current_cache,"sigmoid")
    
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l+1)],current_cache,"relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [205]:
def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters) // 2
    
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)]-learning_rate*grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)]-learning_rate*grads["db" + str(l+1)]
        
    return parameters

In [308]:
def optimize(X, Y, layers_dims, learning_rate = 0.001, num_iterations = 3000, print_cost=False):

    np.random.seed(1)
    costs = []
    parameters = initialize_parameters(layers_dims)

    for i in range(0, num_iterations):
        AL, caches = model_forward(X,parameters)
        print(AL.shape)
        print(caches[2])
        cost = compute_cost(AL,Y)
        print(cost)
        grads = model_backward(AL,Y,caches)
        print(grads[dA2].shape)
        parameters = update_parameters(parameters,grads,learning_rate)
        
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 1000 == 0:
            costs.append(cost)
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

In [309]:
X_train.shape[0]

13

In [310]:
parameters = optimize(X_train, y_train,layers_dims = [X_train.shape[0],100,60,1], num_iterations = 2500, print_cost = True)

(1, 212)
((array([[0.50078871, 0.5008497 , 0.50079979, 0.50134167, 0.50094723,
        0.50089261, 0.50129754, 0.50105554, 0.50057082, 0.50104962,
        0.50101633, 0.50109081, 0.50100225, 0.50108252, 0.50098919,
        0.50134562, 0.50088102, 0.50095605, 0.50072194, 0.50100508,
        0.50086139, 0.50089367, 0.50106948, 0.50085836, 0.50092351,
        0.50110723, 0.50126181, 0.5006388 , 0.50090988, 0.50083372,
        0.50102911, 0.50086632, 0.50086432, 0.50123066, 0.50093919,
        0.50078725, 0.50084877, 0.5013991 , 0.50103721, 0.50108144,
        0.50096674, 0.50095189, 0.50141456, 0.50148129, 0.50116266,
        0.50127175, 0.50092241, 0.50078004, 0.50109846, 0.50091828,
        0.50080414, 0.5012123 , 0.50075148, 0.50082218, 0.5009769 ,
        0.50080614, 0.50093746, 0.50139232, 0.5008506 , 0.50089213,
        0.50081078, 0.5008838 , 0.5006687 , 0.50121934, 0.50094449,
        0.50075551, 0.50125883, 0.50105009, 0.50105226, 0.50119487,
        0.50085616, 0.50118117, 0.500

NameError: name 'dA2' is not defined